In [1]:
import torch
from transformers import PreTrainedModel, PreTrainedTokenizer
from typing import List, Tuple, Dict, Optional
import numpy as np

import json
import pandas as pd
from datasets import load_dataset
from pathlib import Path
import re

from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorc

In [ ]:
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")


def calculate_confidence(logits: List[torch.Tensor], answer_ids: torch.Tensor) -> float:
    """
    Calculate the confidence score (Δ) as specified in the paper.

    Args:
        logits: List of logits for each decoding step
        answer_ids: Tensor of token ids for the answer

    Returns:
        Confidence score (Δ)
    """
    confidence_sum = 0.0
    valid_tokens = 0
    for t, token_id in enumerate(answer_ids):
        if t >= len(logits):
            break
        token_logits = logits[t]
        probs = torch.softmax(token_logits, dim=-1)
        if probs.size(-1) > 1:
            top_2_probs, _ = torch.topk(probs, min(2, probs.size(-1)))
            if top_2_probs.size(-1) > 1:
                confidence_sum += (top_2_probs[-1]
                                   [0] - top_2_probs[-1][1]).item()
            else:
                confidence_sum += 1.0  # Max confidence if there's only one token
        else:
            confidence_sum += 1.0  # Max confidence if there's only one token
        valid_tokens += 1

    return confidence_sum / valid_tokens if valid_tokens > 0 else 0.0


def aggregate_paths_based_on_scores(paths: List[Tuple[str, float]]) -> Tuple[str, float]:
    """Aggregate multiple paths based on their confidence scores."""
    answer_scores = {}
    for answer, delta in paths:
        answer_scores[answer] = answer_scores.get(answer, 0) + delta
    best_answer = max(answer_scores, key=answer_scores.get)
    return best_answer, answer_scores[best_answer]


def cot_decode(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    messages: List[Dict[str, str]],
    k: int = 10,
    num_beams: int = 1,
    max_new_tokens: int = 512,
    temperature: float = 1.0,
    top_p: float = 1.0,
    repetition_penalty: float = 1.0,
    length_penalty: float = 1.0,
    no_repeat_ngram_size: int = 0,
    early_stopping: bool = False,
    aggregate_paths: bool = False,
) -> Tuple[str, float]:
    """
    Implement CoT-decoding for a given chat input.

    Args:
        model: The Hugging Face transformer model.
        tokenizer: The associated tokenizer.
        messages: List of chat messages in the format [{"role": "user", "content": "..."}]
        k: The number of alternative tokens to consider at the first step.
        num_beams: Number of beams for beam search.
        max_new_tokens: Maximum number of new tokens to generate.
        temperature: Sampling temperature.
        top_p: Nucleus sampling probability.
        repetition_penalty: Repetition penalty factor.
        length_penalty: Length penalty factor.
        no_repeat_ngram_size: Size of n-grams to avoid repeating.
        early_stopping: Whether to stop generation when all beams are finished.
        aggregate_paths: Whether to aggregate multiple paths.

    Returns:
        A tuple containing the best path (or aggregated result) and its confidence score.
    """
    device = get_device()
    model.to(device)

    # Use the chat template to format the input
    if tokenizer.chat_template:
        input_text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True)
    else:
        # Fallback for tokenizers without chat templates
        input_text = "\n".join(
            [f"{msg['role']}: {msg['content']}" for msg in messages])
        input_text += "\nassistant:"

    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids).to(device)

    # Set pad_token_id if it's not set
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

    # Get the top-k tokens for the first decoding step
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        first_token_logits = outputs.logits[0, -1, :]
        top_k_logits, top_k_indices = torch.topk(first_token_logits, k)

    paths = []
    for idx in top_k_indices:
        # Generate sequence starting with the selected token
        start_ids = torch.cat(
            [input_ids, idx.unsqueeze(0).unsqueeze(0)], dim=-1)
        start_mask = torch.cat([attention_mask, torch.ones(
            (1, 1), dtype=torch.long, device=device)], dim=-1)

        output = model.generate(
            start_ids,
            attention_mask=start_mask,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            early_stopping=early_stopping,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True,
        )

        generated_sequence = output.sequences[0]
        answer_ids = generated_sequence[len(input_ids[0]):]
        # print(f"answer_ids: {answer_ids}")
        answer_text = tokenizer.decode(answer_ids, skip_special_tokens=True)
        # print(f"answer_text: {answer_text}")

        # Calculate confidence score (Δ)
        confidence = calculate_confidence(output.scores, answer_ids)
        paths.append((answer_text, confidence))

    if aggregate_paths:
        return aggregate_paths_based_on_scores(paths)
    else:
        return max(paths, key=lambda x: x[1])

In [57]:
# name list
# provided by authors

names_string = """Sasha Calle
Annie Murphy
Golshifteh Farahani
Kate Mara
Josh Hartnett
Jennifer Lawrence
Aaron Taylor-Johnson
Rebecca Ferguson
Monica Barbaro
Chris Hemsworth
Wes Anderson
Daniel Portman
Lily-Rose Depp
Myha'la Herrold
Zendaya
Ezra Miller
Olga Kurylenko
Zazie Beetz
Arnold Schwarzenegger
Emilia Clarke
Jess Bush
Clara Rugaard
Molly Gordon
Isabel May
Hailee Steinfeld
Hannah Waddingham
Rory Culkin
Cobie Smulders
Harrison Ford
Tom Cruise
Carol Kane
Alexandra Daddario
Gal Gadot
Tom Holland
Hayley Atwell
Salma Hayek
Ana de Armas
Will Poulter
Anson Mount
Paapa Essiedu
Sam Hargrave
Margot Robbie
Nicolas Cage
Henry Cavill
Juno Temple
Cailee Spaeny
Treat Williams
Alexander Skarsgård
Rebecca Romijn
Monica Dolan
Anya Taylor-Joy
Sophia Lillis
Emmanuelle Vaugier
Aaron Paul
Elliot Page
Robin Tunney
Mike Faist
Tinatin Dalakishvili
Sarah Snook
Jenna Ortega
Zoe Saldana
Anjana Vasan
Ben Mendelsohn
Jeremy Allen White
Ayo Edebiri
Keanu Reeves
Pom Klementieff
Scarlett Johansson
Tornike Gogrichiani
James Cameron
Pedro Pascal
Kaley Cuoco
Samuel L. Jackson
Terri Ivens
Florence Pugh
Shea Whigham
Kingsley Ben-Adir
Michael Keaton
Julian Sands
Christopher Nolan
Tom Hanks
Clint Eastwood
Gabriel Macht
Fabiana Udenio
Tom Bateman
Jack Champion
Jake Gyllenhaal
Leonardo DiCaprio
Jason Schwartzman
Grace Caroline Currey
Sydney Sweeney
Emily Rudd
Samuel Blenkin
James Marsden
Jesse Plemons
Alan Ritchson
Cillian Murphy
Meghan Markle
Tyler Hoechlin
Angelina Jolie
"""
# Christina Chong: excluded due to different information regarding date and year of birth, nothing mentioned on Wikipedia
# One double
# Two added names (Tyler Hoechlin, Angelina Jolie) to get to 100

name_list = names_string.split("\n")
print(len(name_list))

101


In [ ]:
# provided by the authors
# taken from predictions by another model
# manually checked for correctness by us (Wikipedia)
# two added entries due to one double in the original list and one unclear case; now len = 100

year_map = {'Sasha Calle': '1995', 'Annie Murphy': '1986', 'Golshifteh Farahani': '1983', 'Kate Mara': '1983', 'Josh Hartnett': '1978', 'Jennifer Lawrence': '1990', 'Aaron Taylor-Johnson': '1990', 'Rebecca Ferguson': '1983', 'Monica Barbaro': '1990', 'Chris Hemsworth': '1983',

            'Wes Anderson': '1969', 'Daniel Portman': '1992', 'Lily-Rose Depp': '1999', "Myha'la Herrold": '1996', 'Zendaya': '1996', 'Ezra Miller': '1992', 'Olga Kurylenko': '1979', 'Zazie Beetz': '1991', 'Arnold Schwarzenegger': '1947', 'Emilia Clarke': '1986',

            'Jess Bush': '1992', 'Clara Rugaard': '1997', 'Molly Gordon': '1994', 'Isabel May': '2000', 'Hailee Steinfeld': '1996', 'Hannah Waddingham': '1974', 'Christina Chong': '1983', 'Rory Culkin': '1989', 'Cobie Smulders': '1982', 'Harrison Ford': '1942',

            'Tom Cruise': '1962', 'Carol Kane': '1952', 'Alexandra Daddario': '1986', 'Gal Gadot': '1985', 'Tom Holland': '1996', 'Hayley Atwell': '1982', 'Salma Hayek': '1966', 'Ana de Armas': '1988', 'Will Poulter': '1993', 'Anson Mount': '1973',

            'Paapa Essiedu': '1990', 'Sam Hargrave': '1982', 'Margot Robbie': '1990', 'Nicolas Cage': '1964', 'Henry Cavill': '1983', 'Juno Temple': '1989', 'Cailee Spaeny': '1998', 'Treat Williams': '1951',

            'Alexander Skarsgård': '1976', 'Rebecca Romijn': '1972', 'Monica Dolan': '1969', 'Anya Taylor-Joy': '1996', 'Sophia Lillis': '2002', 'Emmanuelle Vaugier': '1976', 'Aaron Paul': '1979', 'Elliot Page': '1987', 'Robin Tunney': '1972', 'Mike Faist': '1992',

            'Tinatin Dalakishvili': '1991', 'Sarah Snook': '1987', 'Jenna Ortega': '2002', 'Zoe Saldana': '1978', 'Anjana Vasan': '1987', 'Ben Mendelsohn': '1969', 'Jeremy Allen White': '1991', 'Ayo Edebiri': '1995', 'Keanu Reeves': '1964', 'Pom Klementieff': '1986',

            'Scarlett Johansson': '1984', 'Tornike Gogrichiani': '1986', 'James Cameron': '1954', 'Pedro Pascal': '1975', 'Kaley Cuoco': '1985', 'Samuel L. Jackson': '1948', 'Terri Ivens': '1967', 'Florence Pugh': '1996', 'Shea Whigham': '1969',

            'Kingsley Ben-Adir': '1986', 'Michael Keaton': '1951', 'Julian Sands': '1958', 'Christopher Nolan': '1970', 'Tom Hanks': '1956', 'Clint Eastwood': '1930', 'Gabriel Macht': '1972', 'Fabiana Udenio': '1964', 'Tom Bateman': '1989', 'Jack Champion': '2004',

            'Jake Gyllenhaal': '1980', 'Leonardo DiCaprio': '1974', 'Jason Schwartzman': '1980', 'Grace Caroline Currey': '1996', 'Sydney Sweeney': '1997', 'Emily Rudd': '1993', 'Samuel Blenkin': '1996', 'James Marsden': '1973', 'Jesse Plemons': '1988', 'Alan Ritchson': '1982',

            'Cillian Murphy': '1976', 'Meghan Markle': '1981', 'Tyler Hoechlin': '1987', 'Angelina Jolie': '1975'}

In [59]:
len(year_map)

101

In [ ]:
for name in name_list:

    # Query and target
    text = "Was " + name + " born in an even or odd year?"
    if name not in year_map:
        print(name)
    else:
        print("in")

in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in



In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
# from the authors, modified
name = "Nicolas Cage"
text = "Q: Was " + name + " born in an even or odd year?\nA:"
if name not in year_map:
    print("Name not in data set.")
year = int(year_map[name])
if year % 2 == 0:
    target = 'even'
else:
    target = 'odd'

In [ ]:
# from the example in the file on GitHub, modified
messages = [
    {"role": "user", "content": f"Q: {text}\nA:"}
]

# Generate the response using CoT decoding
print(f"Using device: {get_device()}")
result, confidence = cot_decode(
    model, tokenizer, messages, aggregate_paths=True, max_new_tokens=200)
print(f"CoT Decoding:\n {result}")

Using device: cuda
CoT Decoding:
 Nicholas Cage was born in January 1974 in Houston, Texas, United States. Texas is an odd-numbered state, so Nicolas Cage would have been born in an odd-numbered year.


In [ ]:
# identifying the actual answer to the question
found = re.findall(r"\Weven\W|\Wodd\W", result, flags=re.IGNORECASE)

# evaluation
if found:
    answer = re.sub(r"\W", "", found[-1])
    if target == answer.lower():
        print("Correct Answer: ", target)
    else:
        print(
            f"Incorrect Answer. Correct Answer: {target}, Answer given: {answer}")
else:
    print("No answer was found.")

Incorrect Answer. Correct Answer: even, Answer given: odd


Iterating through the name list

In [ ]:
name_list = ["Sasha Calle", "Annie Murphy",
             "Golshifteh Farahani", "Kate Mara", "Josh Hartnett"]

In [ ]:
correct: int = 0
no_answer: int = 0
# for analysis, if a sample comes without "boxed" we can retrieve it
indexes_no_answer = []
incorrect: int = 0
indexes_incorrect = []  # for analysis

for name in name_list:

    # Query and target
    text = "Was " + name + " born in an even or odd year?"
    if name not in year_map:
        continue
    year = int(year_map[name])
    if year % 2 == 0:
        target = 'even'
    else:
        target = 'odd'

    # inference
    messages = [
        {"role": "user", "content": f"Q: {text}\nA:"}
    ]
    result, confidence = cot_decode(
        model, tokenizer, messages, aggregate_paths=True, max_new_tokens=200)

    # identifying answer
    found = re.findall(r"\Weven\W|\Wodd\W", result, flags=re.IGNORECASE)

    # evaluation
    if found:
        answer = re.sub(r"\W", "", found[-1])
        if target == answer.lower():
            correct += 1
        else:
            incorrect += 1
            indexes_incorrect.append([name, result, target])
    else:
        no_answer += 1
        indexes_no_answer.append([name, result, target])


# Calculating accuracy:
print(f"Accuracy: {correct/len(name_list) * 100} %")

# Cases were workaround did not work
print(f"No answers found: {no_answer}")

NameError: name 'question_list' is not defined

In [36]:
correct/len(name_list)

0.6

In [38]:
indexes_no_answer

[['Sasha Calle',
  'To determine the year in which Sasha Calle was born, we need to consider the year in which the current day is located on the calendar.\n\nAssuming the current date is the 20th of April 2023 (a common date for a child born in the 21st century):\n\n1. March:\n   - In which month is April 2023 located in the calendar?\n\n2. April:\n   - April is the 4th month of the year.\n\n3. May:\n   - The month after April is May.\n\n4. June:\n   - The month after May is June.\n\n5. July:\n   - The month after July might be August.\n\n6. August:\n   - The month after August is September.\n\n7. September:\n   - The month after September is October.\n\n8. October:\n   - The month after October is November.\n\n9. November:\n   - The month after November is December.\n\n10. December:\n    - The',
  'odd']]

In [40]:
indexes_incorrect

[['Josh Hartnett',
  'According to the information available to me, Josh Hartnett was born in 1981, which is an odd year.',
  'even']]

In [37]:
help_string = "'The correct answer is: Sasha Calle was born in the year 1990. The date of her birth, July 30, 1990, was a leap year, and the year 1990 was also an even number. Odd numbers are numbers ending in 1, 3, 5, 7, or 9, while even numbers are numbers ending in 0, 2, 4, 6, or 8."
help_result = 'odd'

Greedy Decoding

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Single answer

In [41]:
# from the authors, modified
name = "Nicolas Cage"
text = "Was " + name + " born in an even or odd year?"
if name not in year_map:
    print("Name not in data set.")
year = int(year_map[name])
if year % 2 == 0:
    target = 'even'
else:
    target = 'odd'

In [44]:
# encode context the generation is conditioned on
model_inputs = tokenizer(f"Q: {text}\nA:", return_tensors='pt').to("cuda")

# generate 40 new tokens
greedy_output = model.generate(
    **model_inputs,
    num_beams=1,
    do_sample=False,
    max_new_tokens=512
)

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Q: Was Nicolas Cage born in an even or odd year?
A: Even
What is the answer? (Available options:
 -even
 -odd)


In [ ]:
# identifying the actual answer to the question
found = re.findall(r"\Weven\W|\Wodd\W", result, flags=re.IGNORECASE)

# evaluation
if found:
    # for greedy decoding, choose the first occurence of even / odd
    answer = re.sub(r"\W", "", found[0])
    if target == answer.lower():
        print("Correct Answer: ", target)
    else:
        print(
            f"Incorrect Answer. Correct Answer: {target}, Answer given: {answer}")
else:
    print("No answer was found.")

Incorrect Answer. Correct Answer: even, Answer given: odd


In [47]:
name_list

['Sasha Calle',
 'Annie Murphy',
 'Golshifteh Farahani',
 'Kate Mara',
 'Josh Hartnett']

Loop

In [ ]:
correct: int = 0
no_answer: int = 0
# for analysis, if a sample comes without "boxed" we can retrieve it
indexes_no_answer = []
incorrect: int = 0
indexes_incorrect = []  # for analysis

for name in name_list:

    # Query and target
    text = "Was " + name + " born in an even or odd year?"
    if name not in year_map:
        continue
    year = int(year_map[name])
    if year % 2 == 0:
        target = 'even'
    else:
        target = 'odd'

    # inference
    model_inputs = tokenizer(f"Q: {text}\nA:", return_tensors='pt').to("cuda")

    greedy_output = model.generate(
        **model_inputs,
        num_beams=1,
        do_sample=False,
        max_new_tokens=40
    )

    result = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

    # identifying answer
    found = re.findall(r"\Weven\W|\Wodd\W", result, flags=re.IGNORECASE)

    # evaluation
    if found:
        answer = re.sub(r"\W", "", found[-1])
        if target == answer.lower():
            correct += 1
        else:
            incorrect += 1
            indexes_incorrect.append([name, result, target])
    else:
        no_answer += 1
        indexes_no_answer.append([name, result, target])


# Calculating accuracy:
print(f"Accuracy: {correct/len(name_list) * 100} %")

# Cases were workaround did not work
print(f"No answers found: {no_answer}")

Accuracy: 80.0 %
No answers found: 0


In [50]:
indexes_incorrect

[['Kate Mara',
  'Q: Was Kate Mara born in an even or odd year?\nA: Even\nExplanation for the above answer:\nKate Mara was born on February 1, 1964. February is the second month of the year and it falls between January and March. The',
  'odd']]